### Ejemplos de funciones

Partiendo del dataset de bioinformática, vamos a realizar una serie de procesamientos en los que se van a utilizar funciones de Pig así como UDFs de Piggibank y Datafu.

In [1]:
! mkdir -p pig-funciones
import os
os.chdir("pig-funciones")
! pwd

/home/cloudera/HadoopNotebooks/Pig/pig-funciones


In [2]:
%%writefile test.pig
datacleaninternal = load 'bioinformatica.datacleaninternal' using org.apache.hive.hcatalog.pig.HCatLoader();

describe datacleaninternal;

Overwriting test.pig


In [3]:
! pig -f test.pig

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-20 00:05:31,426 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-20 00:05:31,426 [main] INFO  org.apache.pig.Main - Logging error messages to: /home/cloudera/HadoopNotebooks/Pig/pig-funciones/pig_1516435531410.log
2018-01-20 00:05:32,322 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-20 00:05:32,428 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2018-01-20 00:05:32,428 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:05:32,428 [main] INFO  org.apache.

In [13]:
%%writefile test.pig
-- dividimos la relacion datacleaninternal en varias relaciones segun 
-- el valor del campo gsm19023, 
-- para luego calcular la media del campo gsd19026 en cada grupo

datacleaninternal = load 'bioinformatica.datacleaninternal' using org.apache.hive.hcatalog.pig.HCatLoader();

split datacleaninternal into gsm19023bajo if gsm19023 > 100 and gsm19023 < 200, 
gsm19023medio if gsm19023 > 200 and gsm19023 < 500, 
gsm19023alto if gsm19023 > 500;


-- necesario agrupar todos los registros en una bag ya que AVG trabaja 
-- sobre bags
gsm19023bajogroup= group  gsm19023bajo all; 
gsm19023bajomedia = foreach gsm19023bajogroup generate AVG(gsm19023bajo.gsd19026); 

STORE gsm19023bajomedia INTO 'gsm19023bajomediaC';


-- necesario agrupar todos los registros en una bag ya que AVG trabaja sobre bags
gsm19023mediogroup= group  gsm19023medio all; 
gsm19023mediomedia = foreach gsm19023mediogroup generate AVG(gsm19023medio.gsd19026); 
--dump gsm19023mediomedia;

STORE gsm19023mediomedia INTO 'gsm19023mediomediaC';


-- necesario agrupar todos los registros en una bag ya que AVG trabaja sobre bags
gsm19023altogroup= group  gsm19023alto all; 
gsm19023altomedia = foreach gsm19023altogroup generate AVG(gsm19023alto.gsd19026); 
--dump gsm19023altomedia;

STORE gsm19023altomedia INTO 'gsm19023altomediaC';



Overwriting test.pig


In [14]:
! pig -f test.pig

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-20 00:29:08,655 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-20 00:29:08,656 [main] INFO  org.apache.pig.Main - Logging error messages to: /home/cloudera/HadoopNotebooks/Pig/pig-funciones/pig_1516436948640.log
2018-01-20 00:29:09,514 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-20 00:29:09,615 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2018-01-20 00:29:09,615 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:29:09,615 [main] INFO  org.apache.

In [10]:
! hadoop fs -cat /user/cloudera/gsm19023bajomediaC/part-r-00000
! hadoop fs -cat /user/cloudera/gsm19023mediomediaC/part-r-00000
! hadoop fs -cat /user/cloudera/gsm19023altomediaC/part-r-00000

170.78338433568118


Filtraremos las entradas de datacleaninternal para quedarnos con aquellas que tengan genetitle == 'stomatin' y a estas les aplicamos algunas funciones de string

In [19]:
%%writefile test.pig

datacleaninternal = load 'bioinformatica.datacleaninternal' using org.apache.hive.hcatalog.pig.HCatLoader();
stomatin = filter datacleaninternal by genetitle == 'stomatin';

--dump stomatin;


-- nos quedamos solamente con el genetitle, y le cambiamos 
-- algunos caracteres
stomatin2 = foreach stomatin generate REPLACE(genetitle, 'tomatin', 'tomate') AS genetitlenew;

dump stomatin2;
describe stomatin2;



Overwriting test.pig


In [20]:
! pig -f test.pig -x local

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-20 00:35:44,838 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-20 00:35:44,838 [main] INFO  org.apache.pig.Main - Logging error messages to: /home/cloudera/HadoopNotebooks/Pig/pig-funciones/pig_1516437344826.log
2018-01-20 00:35:44,849 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2018-01-20 00:35:45,285 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-20 00:35:45,345 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:35:45,345 [main] INFO  org.apache.hadoop.conf.Co

2018-01-20 00:35:47,396 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2018-01-20 00:35:47,423 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.task.id is deprecated. Instead, use mapreduce.task.attempt.id
2018-01-20 00:35:47,887 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.input.dir is deprecated. Instead, use mapreduce.input.fileinputformat.inputdir
2018-01-20 00:35:48,041 [JobControl] INFO  org.apache.hadoop.mapred.FileInputFormat - Total input paths to process : 5
2018-01-20 00:35:48,051 [JobControl] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths (combined) to process : 1
2018-01-20 00:35:48,082 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2018-01-20 00:35:48,090 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.

2018-01-20 00:35:49,321 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecordReader - Current split being processed org.apache.hive.hcatalog.mapreduce.HCatSplit@43d8842b
2018-01-20 00:35:49,325 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hive.hcatalog.mapreduce.InternalUtil - Initializing org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe with properties {name=bioinformatica.datacleaninternal, numFiles=5, field.delim=	, columns.types=string,string,float,float,float,float,string,string,smallint,string,string,string,string, serialization.format=	, columns=idref,ident,gsm19023,gsd19024,gsd19025,gsd19026,genetitle,genesymbol,geneid,unigenetitle,unigenesymbol,unigeneid,nucleotidetitle, serialization.lib=org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, COLUMN_STATS_ACCURATE=true, totalSize=9978930, serialization.null.format=\N, transient_lastDdlTime=1515747829}
2018-01-20 00:35:49,328 [LocalJobRunner Map Task Executor

In [21]:
%%writefile test.pig

REGISTER /usr/lib/pig/piggybank.jar;
DEFINE Reverse org.apache.pig.piggybank.evaluation.string.Reverse();

datacleaninternal = load 'bioinformatica.datacleaninternal' using org.apache.hive.hcatalog.pig.HCatLoader();
stomatin = filter datacleaninternal by genetitle == 'stomatin';

-- le damos la vuelta al genetitlenew (stomate -> etamots)
stomatin3 = FOREACH stomatin GENERATE Reverse(genetitle) AS reversed_genetitle;
dump stomatin3;

Overwriting test.pig


In [22]:
! pig -f test.pig -x local

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-20 00:39:06,724 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-20 00:39:06,725 [main] INFO  org.apache.pig.Main - Logging error messages to: /home/cloudera/HadoopNotebooks/Pig/pig-funciones/pig_1516437546710.log
2018-01-20 00:39:06,736 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2018-01-20 00:39:07,254 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-20 00:39:07,325 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:39:07,326 [main] INFO  org.apache.hadoop.conf.Co

2018-01-20 00:39:09,452 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2018-01-20 00:39:09,474 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.task.id is deprecated. Instead, use mapreduce.task.attempt.id
2018-01-20 00:39:09,917 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.input.dir is deprecated. Instead, use mapreduce.input.fileinputformat.inputdir
2018-01-20 00:39:10,040 [JobControl] INFO  org.apache.hadoop.mapred.FileInputFormat - Total input paths to process : 5
2018-01-20 00:39:10,048 [JobControl] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths (combined) to process : 1
2018-01-20 00:39:10,076 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2018-01-20 00:39:10,083 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.

2018-01-20 00:39:11,250 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecordReader - Current split being processed org.apache.hive.hcatalog.mapreduce.HCatSplit@3df020ff
2018-01-20 00:39:11,254 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hive.hcatalog.mapreduce.InternalUtil - Initializing org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe with properties {name=bioinformatica.datacleaninternal, numFiles=5, field.delim=	, columns.types=string,string,float,float,float,float,string,string,smallint,string,string,string,string, serialization.format=	, columns=idref,ident,gsm19023,gsd19024,gsd19025,gsd19026,genetitle,genesymbol,geneid,unigenetitle,unigenesymbol,unigeneid,nucleotidetitle, serialization.lib=org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, COLUMN_STATS_ACCURATE=true, totalSize=9978930, serialization.null.format=\N, transient_lastDdlTime=1515747829}
2018-01-20 00:39:11,258 [LocalJobRunner Map Task Executor

In [25]:
%%writefile test.pig

datacleaninternal = load 'bioinformatica.datacleaninternal' using org.apache.hive.hcatalog.pig.HCatLoader();
stomatin = filter datacleaninternal by genetitle == 'stomatin';

-- dividimos el genetitle utilizando la letra t como delimitador. Obtenemos una tupla dentro de otra tupla ((s,oma,in))
stomatin4 = FOREACH stomatin GENERATE STRSPLIT(genetitle, 't') AS split_genetitle;
dump stomatin4;

-- FLATTEN saca los componentes fuera de la tupla (s,oma,in)
stomatin5 = FOREACH stomatin4 GENERATE FLATTEN(split_genetitle) AS split_genetitle;
dump stomatin5;


Overwriting test.pig


In [26]:
! pig -f test.pig -x local

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-20 00:43:06,110 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-20 00:43:06,111 [main] INFO  org.apache.pig.Main - Logging error messages to: /home/cloudera/HadoopNotebooks/Pig/pig-funciones/pig_1516437786098.log
2018-01-20 00:43:06,121 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2018-01-20 00:43:06,549 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-20 00:43:06,609 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:43:06,610 [main] INFO  org.apache.hadoop.conf.Co

2018-01-20 00:43:08,628 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2018-01-20 00:43:08,656 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.task.id is deprecated. Instead, use mapreduce.task.attempt.id
2018-01-20 00:43:09,144 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.input.dir is deprecated. Instead, use mapreduce.input.fileinputformat.inputdir
2018-01-20 00:43:09,289 [JobControl] INFO  org.apache.hadoop.mapred.FileInputFormat - Total input paths to process : 5
2018-01-20 00:43:09,297 [JobControl] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths (combined) to process : 1
2018-01-20 00:43:09,325 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2018-01-20 00:43:09,332 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.

2018-01-20 00:43:10,500 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecordReader - Current split being processed org.apache.hive.hcatalog.mapreduce.HCatSplit@27ebc1f7
2018-01-20 00:43:10,503 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hive.hcatalog.mapreduce.InternalUtil - Initializing org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe with properties {name=bioinformatica.datacleaninternal, numFiles=5, field.delim=	, columns.types=string,string,float,float,float,float,string,string,smallint,string,string,string,string, serialization.format=	, columns=idref,ident,gsm19023,gsd19024,gsd19025,gsd19026,genetitle,genesymbol,geneid,unigenetitle,unigenesymbol,unigeneid,nucleotidetitle, serialization.lib=org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, COLUMN_STATS_ACCURATE=true, totalSize=9978930, serialization.null.format=\N, transient_lastDdlTime=1515747829}
2018-01-20 00:43:10,507 [LocalJobRunner Map Task Executor

2018-01-20 00:43:27,811 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:43:27,812 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2018-01-20 00:43:27,824 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.JobControlCompiler - Setting up single store job
2018-01-20 00:43:27,825 [main] INFO  org.apache.pig.data.SchemaTupleFrontend - Key [pig.schematuple] is false, will not generate code.
2018-01-20 00:43:27,825 [main] INFO  org.apache.pig.data.SchemaTupleFrontend - Starting process to move generated code to distributed cache
2018-01-20 00:43:27,825 [main] INFO  org.apache.pig.data.SchemaTupleFrontend - Distributed cache not supported or needed in local mode. Setting key [pig.schematuple.local.dir] with code temp directory: /tmp/1516437807824-0
2018-01-20 00:43:27,845 [main] INFO  org.apach

2018-01-20 00:43:28,229 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecordReader - Current split being processed org.apache.hive.hcatalog.mapreduce.HCatSplit@482092f4
2018-01-20 00:43:28,232 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hive.hcatalog.mapreduce.InternalUtil - Initializing org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe with properties {name=bioinformatica.datacleaninternal, numFiles=5, field.delim=	, columns.types=string,string,float,float,float,float,string,string,smallint,string,string,string,string, serialization.format=	, columns=idref,ident,gsm19023,gsd19024,gsd19025,gsd19026,genetitle,genesymbol,geneid,unigenetitle,unigenesymbol,unigeneid,nucleotidetitle, serialization.lib=org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, COLUMN_STATS_ACCURATE=true, totalSize=9978930, serialization.null.format=\N, transient_lastDdlTime=1515747829}
2018-01-20 00:43:28,235 [LocalJobRunner Map Task Executor

In [34]:
%%writefile test.pig

datacleaninternal = load 'bioinformatica.datacleaninternal' using org.apache.hive.hcatalog.pig.HCatLoader();
stomatin = filter datacleaninternal by genetitle == 'stomatin';

-- (Mus musculus integral membrane phosphoprotein band 7.2b (Epb72) mRNA, complete cds)
stomatinnucle = FOREACH stomatin GENERATE nucleotidetitle;
--dump stomatinnucle;


-- TOKENIZE separa el string que toma como parametro utilizando los blancos
--({(Mus),(musculus),(integral),(membrane),(phosphoprotein),(band),(7.2b),(Epb72),(mRNA),(complete),(cds)})
stomatintoken = FOREACH stomatin GENERATE TOKENIZE(nucleotidetitle) AS split_genetitle;
--dump stomatintoken;


-- FLATTEN saca los componentes fuera de la tupla 
--(Mus)
--(musculus)
--...
stomatin5 = FOREACH stomatintoken GENERATE FLATTEN(split_genetitle) AS flat_genetitle;
dump stomatin5;


Overwriting test.pig


In [35]:
! pig -f test.pig -x local

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-20 00:50:29,077 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-20 00:50:29,077 [main] INFO  org.apache.pig.Main - Logging error messages to: /home/cloudera/HadoopNotebooks/Pig/pig-funciones/pig_1516438229066.log
2018-01-20 00:50:29,088 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2018-01-20 00:50:29,520 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-20 00:50:29,579 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:50:29,579 [main] INFO  org.apache.hadoop.conf.Co

2018-01-20 00:50:31,607 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2018-01-20 00:50:31,627 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.task.id is deprecated. Instead, use mapreduce.task.attempt.id
2018-01-20 00:50:32,121 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.input.dir is deprecated. Instead, use mapreduce.input.fileinputformat.inputdir
2018-01-20 00:50:32,259 [JobControl] INFO  org.apache.hadoop.mapred.FileInputFormat - Total input paths to process : 5
2018-01-20 00:50:32,267 [JobControl] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths (combined) to process : 1
2018-01-20 00:50:32,297 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2018-01-20 00:50:32,305 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.

2018-01-20 00:50:33,461 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecordReader - Current split being processed org.apache.hive.hcatalog.mapreduce.HCatSplit@3111786c
2018-01-20 00:50:33,464 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hive.hcatalog.mapreduce.InternalUtil - Initializing org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe with properties {name=bioinformatica.datacleaninternal, numFiles=5, field.delim=	, columns.types=string,string,float,float,float,float,string,string,smallint,string,string,string,string, serialization.format=	, columns=idref,ident,gsm19023,gsd19024,gsd19025,gsd19026,genetitle,genesymbol,geneid,unigenetitle,unigenesymbol,unigeneid,nucleotidetitle, serialization.lib=org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, COLUMN_STATS_ACCURATE=true, totalSize=9978930, serialization.null.format=\N, transient_lastDdlTime=1515747829}
2018-01-20 00:50:33,467 [LocalJobRunner Map Task Executor

Trabajo con grupos. Devuelve los items agrupados por genetitle, y dentro de cada genetitle, deseamos saber los valores de gsd19024, y la cuenta de valores distintos de gsd19024.

Ejemplo de salida:
((3-hydroxy-3-methylglutaryl-Coenzyme A reductase,{(80.6),(1025.4)},2))

donde:

3-hydroxy-3-methylglutaryl-Coenzyme A reductase es el genetitle

80.6 y 1025.4 son los valores que el campo gsd19024 tiene para los items cuyo genetitle es el de arriba.

In [38]:
%%writefile test.pig
datacleaninternal = load 'bioinformatica.datacleaninternal' using org.apache.hive.hcatalog.pig.HCatLoader();
genetitlegroup = group datacleaninternal by genetitle;
--describe genetitlegroup;

--dump genetitlegroup;


-- genetitlegroup: {group: chararray,datacleaninternal: {(idref: chararray,ident: chararray,gsm19023: float,gsd19024: float,gsd19025: float,gsd19026: float,genetitle: chararray,genesymbol: chararray,geneid: int,unigenetitle: chararray,unigenesymbol: chararray,unigeneid: chararray,nucleotidetitle: chararray)}}
    
genetitlegroup_ordered = FOREACH genetitlegroup {
    ordered_data = order datacleaninternal by gsd19024 asc; 
    generate (group, ordered_data.gsd19024, SIZE(ordered_data.gsd19024));
}; 

dump genetitlegroup_ordered;

Overwriting test.pig


In [39]:
! pig -f test.pig -x local

log4j:WARN No appenders could be found for logger (org.apache.hadoop.util.Shell).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.
2018-01-20 00:57:39,697 [main] INFO  org.apache.pig.Main - Apache Pig version 0.12.0-cdh5.12.0 (rexported) compiled Jun 29 2017, 04:34:31
2018-01-20 00:57:39,697 [main] INFO  org.apache.pig.Main - Logging error messages to: /home/cloudera/HadoopNotebooks/Pig/pig-funciones/pig_1516438659685.log
2018-01-20 00:57:39,710 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2018-01-20 00:57:40,126 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /home/cloudera/.pigbootup not found
2018-01-20 00:57:40,194 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:57:40,194 [main] INFO  org.apache.hadoop.conf.Co

2018-01-20 00:57:42,677 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 1 map-reduce job(s) waiting for submission.
2018-01-20 00:57:42,677 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, use mapreduce.jobtracker.http.address
2018-01-20 00:57:42,677 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2018-01-20 00:57:42,687 [JobControl] INFO  org.apache.hadoop.metrics.jvm.JvmMetrics - Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
2018-01-20 00:57:42,705 [JobControl] INFO  org.apache.hadoop.conf.Configuration.deprecation - fs.default.name is deprecated. Instead, use fs.defaultFS
2018-01-20 00:57:42,845 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#se

2018-01-20 00:57:44,564 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecordReader - Current split being processed org.apache.hive.hcatalog.mapreduce.HCatSplit@611a22e2
2018-01-20 00:57:44,567 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hive.hcatalog.mapreduce.InternalUtil - Initializing org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe with properties {name=bioinformatica.datacleaninternal, numFiles=5, field.delim=	, columns.types=string,string,float,float,float,float,string,string,smallint,string,string,string,string, serialization.format=	, columns=idref,ident,gsm19023,gsd19024,gsd19025,gsd19026,genetitle,genesymbol,geneid,unigenetitle,unigenesymbol,unigeneid,nucleotidetitle, serialization.lib=org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, COLUMN_STATS_ACCURATE=true, totalSize=9978930, serialization.null.format=\N, transient_lastDdlTime=1515747829}
2018-01-20 00:57:44,570 [LocalJobRunner Map Task Executor

2018-01-20 00:57:45,787 [pool-7-thread-1] WARN  org.apache.pig.data.SchemaTupleBackend - SchemaTupleBackend has already been initialized
2018-01-20 00:57:45,797 [pool-7-thread-1] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigMapReduce$Reduce - Aliases being processed per job phase (AliasName[line,offset]): M: datacleaninternal[1,20],genetitlegroup[2,17] C:  R: genetitlegroup_ordered[10,25]
2018-01-20 00:57:46,575 [pool-7-thread-1] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local877467737_0001_r_000000_0 is done. And is in the process of committing
2018-01-20 00:57:46,578 [pool-7-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 1 / 1 copied.
2018-01-20 00:57:46,578 [pool-7-thread-1] INFO  org.apache.hadoop.mapred.Task - Task attempt_local877467737_0001_r_000000_0 is allowed to commit now
2018-01-20 00:57:46,580 [pool-7-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local877467737_0001_r_

((CDC-like kinase 1,{(5794.8),(5794.8),(5794.8),(5794.8),(5794.8)},5))
((CDC-like kinase 2,{(1908.6),(1908.6),(1908.6),(1908.6),(1908.6)},5))
((CDC-like kinase 3,{(3086.1),(3086.1),(3086.1),(3086.1),(3086.1)},5))
((CDC16 cell division cycle 16,{(1349.1),(1349.1),(1349.1),(1349.1),(1349.1)},5))
((CDC23 cell division cycle 23,{(998.0),(998.0),(998.0),(998.0),(998.0)},5))
((CDC42 effector protein (Rho GTPase binding) 4,{(688.2),(688.2),(688.2),(688.2),(688.2)},5))
((CDC42 small effector 1,{(2974.2),(2974.2),(2974.2),(2974.2),(2974.2)},5))
((CDC42 small effector 2,{(2601.9),(2601.9),(2601.9),(2601.9),(2601.9)},5))
((CDGSH iron sulfur domain 1,{(62.0),(62.0),(62.0),(62.0),(62.0),(10199.6),(10199.6),(10199.6),(10199.6),(10199.6)},10))
((CDK2 (cyclin-dependent kinase 2)-associated protein 1,{(6294.0),(6294.0),(6294.0),(6294.0),(6294.0)},5))
((CDK2 associated, cullin domain 1,{(2107.2),(2107.2),(2107.2),(2107.2),(2107.2)},5))
((CDK2-associated protein 2,{(1952.6),(1952.6),(1952.6),(1952.6),(19

...

((chemokine (C-C motif) ligand 7,{(241.4),(241.4),(241.4),(241.4),(241.4)},5))
((chemokine (C-C motif) ligand 8,{(1123.8),(1123.8),(1123.8),(1123.8),(1123.8)},5))
((chemokine (C-C motif) ligand 9,{(941.9),(941.9),(941.9),(941.9),(941.9)},5))
((chemokine (C-C motif) receptor 1,{(125.4),(125.4),(125.4),(125.4),(125.4)},5))
((chemokine (C-C motif) receptor 1-like 1,{(160.6),(160.6),(160.6),(160.6),(160.6)},5))
((chemokine (C-C motif) receptor 2,{(723.8),(723.8),(723.8),(723.8),(723.8)},5))
((chemokine (C-C motif) receptor 3,{(31.6),(31.6),(31.6),(31.6),(31.6)},5))
((chemokine (C-C motif) receptor 4,{(55.0),(55.0),(55.0),(55.0),(55.0)},5))
((chemokine (C-C motif) receptor 5,{(42.2),(42.2),(42.2),(42.2),(42.2),(124.5),(124.5),(124.5),(124.5),(124.5)},10))
((chemokine (C-C motif) receptor 6,{(1522.1),(1522.1),(1522.1),(1522.1),(1522.1)},5))
((chemokine (C-C motif) receptor 7,{(628.8),(628.8),(628.8),(628.8),(628.8)},5))
((chemokine (C-C motif) receptor 8,{(60.7),(60.7),(60.7),(60.7),(60.7)},

((lamin B2,{(637.8),(637.8),(637.8),(637.8),(637.8)},5))
((laminin B1,{(845.1),(845.1),(845.1),(845.1),(845.1)},5))
((laminin, alpha 1,{(195.0),(195.0),(195.0),(195.0),(195.0)},5))
((laminin, alpha 2,{(1915.5),(1915.5),(1915.5),(1915.5),(1915.5)},5))
((laminin, alpha 3,{(502.2),(502.2),(502.2),(502.2),(502.2)},5))
((laminin, alpha 4,{(1708.0),(1708.0),(1708.0),(1708.0),(1708.0)},5))
((laminin, alpha 5,{(916.7),(916.7),(916.7),(916.7),(916.7)},5))
((laminin, beta 2,{(1386.9),(1386.9),(1386.9),(1386.9),(1386.9)},5))
((laminin, beta 3,{(769.1),(769.1),(769.1),(769.1),(769.1)},5))
((laminin, gamma 1,{(53.9),(53.9),(53.9),(53.9),(53.9)},5))
((laminin, gamma 2,{(1298.8),(1298.8),(1298.8),(1298.8),(1298.8)},5))
((lanosterol synthase,{(375.7),(375.7),(375.7),(375.7),(375.7)},5))
((large subunit GTPase 1 homolog (S. cerevisiae),{(674.3),(674.3),(674.3),(674.3),(674.3)},5))
((large tumor suppressor 2,{(5226.8),(5226.8),(5226.8),(5226.8),(5226.8)},5))
((late endosomal/lysosomal adaptor, MAPK and 

((predicted gene 15448///predicted gene 14548///predicted pseudogene 10693///paired Ig-like receptor B///paired-Ig-like receptor A7///paired-Ig-like receptor A6///paired-Ig-like receptor A4///leukocyte immunoglobulin-like receptor, subfamily A (with TM domain), member 6///paired-Ig-like receptor A2///paired-Ig-like receptor A11///paired-Ig-like receptor A1,{(844.9),(844.9),(844.9),(844.9),(844.9)},5))
((predicted gene 1673,{(647.5),(647.5),(647.5),(647.5),(647.5),(2300.7),(2300.7),(2300.7),(2300.7),(2300.7)},10))
((predicted gene 20605///sin3 associated polypeptide///leucine-rich repeats and calponin homology (CH) domain containing 4,{(1327.9),(1327.9),(1327.9),(1327.9),(1327.9),(3835.0),(3835.0),(3835.0),(3835.0),(3835.0)},10))
((predicted gene 2397///RIKEN cDNA C130026I21 gene,{(312.2),(312.2),(312.2),(312.2),(312.2)},5))
((predicted gene 2436///protein phosphatase 2, regulatory subunit B'', gamma,{(392.3),(392.3),(392.3),(392.3),(392.3)},5))
((predicted gene 2682,{(27.4),(27.4),(27.

((src homology 2 domain-containing transforming protein D,{(150.7),(150.7),(150.7),(150.7),(150.7)},5))
((src homology 2 domain-containing transforming protein E,{(729.9),(729.9),(729.9),(729.9),(729.9)},5))
((src homology three (SH3) and cysteine rich domain,{(305.0),(305.0),(305.0),(305.0),(305.0)},5))
((src-like adaptor,{(2421.3),(2421.3),(2421.3),(2421.3),(2421.3)},5))
((src-related kinase lacking C-terminal regulatory tyrosine and N-terminal myristylation sites,{(45.7),(45.7),(45.7),(45.7),(45.7)},5))
((stabilin 1,{(811.5),(811.5),(811.5),(811.5),(811.5)},5))
((stannin,{(30.8),(30.8),(30.8),(30.8),(30.8)},5))
((stanniocalcin 1,{(50.4),(50.4),(50.4),(50.4),(50.4)},5))
((stanniocalcin 2,{(592.0),(592.0),(592.0),(592.0),(592.0)},5))
((staphylococcal nuclease and tudor domain containing 1,{(3057.6),(3057.6),(3057.6),(3057.6),(3057.6)},5))
((starch binding domain 1,{(480.9),(480.9),(480.9),(480.9),(480.9)},5))
((stathmin 1,{(43.0),(43.0),(43.0),(43.0),(43.0),(26746.6),(26746.6),(26746.

2018-01-20 00:58:01,746 [Thread-4] INFO  hive.metastore - Closed a connection to metastore, current connections: 0
